In [1]:
import os
import tensorflow as tf
import numpy as np
import itertools
import matplotlib.pyplot as plt
import gc
import pandas as pd
from datetime import datetime
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv("..\\..\\..\\..\\Dataset\\02-03-2018.csv", low_memory = False)

In [3]:
df

,Dst Port,Fwd PSH Flags,Protocol,Bwd PSH Flags,Flow Duration,Fwd URG Flags,Tot Fwd Pkts,Bwd URG Flags,Tot Bwd Pkts,FIN Flag Cnt,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,continuous,0 1,continuous,discrete,continuous,discrete,continuous,discrete,continuous,0 1,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,443.0,0,6.0,0,141385.0,0,9.0,0,7.0,0,...,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,49684.0,1,6.0,0,281.0,0,2.0,0,1.0,0,...,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,443.0,0,6.0,0,279824.0,0,11.0,0,15.0,0,...,5.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044522,3389.0,0,6.0,0,3982183.0,0,14.0,0,8.0,0,...,7.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1044523,3389.0,0,6.0,0,3802316.0,0,14.0,0,8.0,0,...,7.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1044524,3389.0,0,6.0,0,4004239.0,0,14.0,0,8.0,0,...,7.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1044525,3389.0,0,6.0,0,3998435.0,0,14.0,0,8.0,0,...,7.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df = df.drop([0,1])

In [5]:
input_label = np.array(df.loc[:, df.columns != "Label"]).astype(np.float)

In [6]:
output_label = np.array(df["Label"])

In [7]:
out = []
for o in output_label:
    if(o == "Benign"):out.append(0)
    else: out.append(1)
output_label = out

In [8]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(input_label)
input_label = scaler.transform(input_label)

<h2>PCA</h2>

In [9]:
inp = shuffle(input_label)

In [10]:
pca = PCA(n_components=18)

In [11]:
pca.fit(inp)

PCA(n_components=18)

<h2>cross validation</h2>

In [12]:
input_label = pca.transform(input_label)

In [13]:
inp = []
out = []
num = 0
for i in range(len(input_label) - 20 + 1):
    aux = []
    for j in range(i, i + 20):
        aux.append(input_label[j])
    inp.append(aux)
    out.append(output_label[i + 20 - 1])

In [14]:
input_label = np.array(inp)
output_label = np.array(out)

In [15]:
inp = None
out = None
gc.collect()

30

In [16]:
def createModel():
    model = keras.Sequential([
        layers.Input(shape = (20,18)),
        layers.LSTM(units = 16, return_sequences = True, use_bias = True),
        layers.LSTM(units = 8, return_sequences = False, use_bias = True),
        layers.Dense(units = 2, activation = "softmax")
    ])
    model.compile(optimizer= keras.optimizers.Adam(learning_rate= 0.00025), loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    return model

In [17]:
skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state=1)

In [18]:
confusion_matrixs = []
roc_curvs = []

In [ ]:
for i, (train, test) in enumerate(skf.split(input_label, output_label)):
    inp_train, out_train = np.array(input_label)[train], np.array(output_label)[train]
    inp_test, out_test = np.array(input_label)[test], np.array(output_label)[test]
    model = createModel()
    model.fit(x = inp_train, y = out_train, validation_split= 0.1, epochs = 10, shuffle = True,verbose = 0)
    res = np.array([np.argmax(resu) for resu in model.predict(inp_test)])
    confusion_matrixs.append(confusion_matrix(out_test, res))
    fpr, tpr, _ = roc_curve(out_test,  res)
    auc = roc_auc_score(out_test, res)
    roc_curvs.append([fpr, tpr, auc])
    print("Modelo " + str(i) + ", auc = " + str(auc))

Epoch 1/10
14210/26440 [===============>..............] - ETA: 1:58 - loss: 0.0604 - accuracy: 0.9842

<h2>Roc Curves</h2>

In [ ]:
cores = ["blue", "orange", "green", "red", "purple", "brown", "pink", "gray", "olive", "cyan"]
for i in range(10):
    plt.plot(roc_curvs[i][0],roc_curvs[i][1],label="curva " + str(i) + ", auc=" + str(roc_curvs[i][2]), c = cores[i])
plt.legend(loc=4)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

In [ ]:
total_conv_matrix = [[0,0],[0,0]]
for cov in confusion_matrixs:
    total_conv_matrix[0][0] += cov[0][0] 
    total_conv_matrix[0][1] += cov[0][1]
    total_conv_matrix[1][0] += cov[1][0]
    total_conv_matrix[1][1] += cov[1][1]


In [ ]:
def plot_confusion_matrix(cm, classes, normaliza = False, title = "Confusion matrix", cmap = plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normaliza:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")
    
    print(cm)
    
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
        horizontalalignment="center",
        color="white" if cm[i,j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
labels = ["Benign", "Bot"]
plot_confusion_matrix(cm = np.array(total_conv_matrix), classes = labels, title = "Bot IDS")